# Generic tensor contractions

This first part is Almost literally copied from  G. Evenbly tensors.net tutorial 1

## Defining generic tensors  
Here we will start studying generic tensor networks and their contraction.
Part of any tensor network study is to understand how to contract some of the parts of the tensor networks, and eventually simplify them. 

The first task is to initialize tensors, we typically did this by creating vectors and reshaping them into higher dimensional tensors, today we will start with those higher dimensional tensors.
Let' s create a random tensor with three legs (an order 4 tensor) with size 2, 3, 4, 5 and call it A, remember we always work with complex tensors in quantum mechanics
and an order 3 tensor with size 4 5 6 and call it B

Then we create an identity matrix that is 5 by 5 and call it C

Other special tensors are those made by all ones, that clearly can be multiplied by an arbitrary random complex constant, thus obtaining a tensor where each element is the same, create one that has order 4 and dimension 2, 4, 2, 4, create it and call it D

Finally when a tensor only has few non-zero elements, one can create a tensor made of all zeros and fill the desider elements. E.g. create a tensor with order 2 made of zero and fill the first element with a random complex number

Now we can reorder the legs (permuting them which incurs in a computational cost proportional to the size of the tensor) or grouping or splitting the legs, which does not have a relevant computational cost (for large tensors) since it only changes the labels used to address the elements ![reshape_permute](../pictures/reshape_permute.png "from G. Evenbly")
For example implement the above permutation and reshaping  for the tensor A  and B defined above

## Tensor contractions

We now enter the realm of tensor contractions. First of all  remeber that contracting two tensors means summing the product of the tensor elements, it is a generalization of matrix multiplication

$ M^i_k =\sum _j A^i_j * B^j_k$, in the theory material you have gone through the diagramatic (or Penrose) notation for such operations. So let's put them in practice here. 

We will start by contracting two tensors. This can be done in several ways.
Define a new tensor $B$ with order 4 and dimensions 3,4,2,5
Now contract it with the $A$ tensor defined above on the second and fifth leg
![Contraction between two tensors](../pictures/two_tensors_contract.png "For G. Evenbly")

First compute it using for loops

Now repeat the same operation by transforming the two tensors into matrix and then performing a matrix multiplication, call the resulting tensor $\tilde{C}$ and compare that the two methods provide the same result.
![Contraction as matrix multiplication](../pictures/contraction_as_matrix_multiplication.png "For G. Evenbly")


## Computational cost of tensor contractions. 

As you have seen in the theory material, contracting tensors comes at a cost, here there is the summary of that cost is in this picture from tensors.net 
![Cost of contraction](../pictures/contraction_cost.png "from G. Evenbly")


## More than 2 tensors 
When your network to contract include more than two tensors it is computationally advantageous to break the contraction into pairwise contractions. For example if you need to contract three tensors, and you do it in a single shot (by using for loops) you incur into a higher computational cost. Try it below follwing the diagarm. Contract it with for loops and by sequence of matrix multiplications.
![three_tensors](../pictures/three_tensors.png "from G. Evenbly")


In general the cost of the contraction of a tensor network depends on the order of the contraction. The picture below from tensors.net provide an explicit example 
![Cost of contraction](../pictures/order_cost.png "from G. Evenbly")


As an exercise find the optimal contraction sequence of this diagarm (from tensors.net) and write the code the performs the optimized contraction transforming the tensors to matrices and multiplying them.
 ![reshape_permute](../pictures/exercice.png "from G. Evenbly")
  Initialize the tensors as random tensors whose legs all share the same dimension $d=10$ 
  use the following convention for the ordering of indices
   ![reshape_permute](../pictures/oredering_indices.png "from G. Evenbly")


## Libraries to perform tensor contractions

We are now in position to use libraries 
Let's start by redefining the tensors, aslo since here we are interested in optimizing the contraction also import the time package and the matplotlib to perform 


We now retake the computation of the last class and check that the leading cost is actually  $d^5$
 , we define the function contraction_by_hand, that take the five tensors and perform the contraction using only transpose and reshape as last time,

Now repeat the contraction for several d in the interval 2- collect the execution time and plot then in a log-log plot. (Why do we use a log-log plot?) On the x-axis put the bond dimension  d  and on the y-axis put the execution time  t
 

Now we define the same contraction but using ncon, and check that the result and scaling are the same than the contraction by hand (also check the intermediate steps).

### Norm of an MPS
Define the MPS by a list of N three leg tensors (the boundary tensors only have 2 legs) with size $D,D,d$, worder their leg as in the drawing,
Define a function that computes the norm of the MPS, 

and check by runnning it for several $D$ in the range $400 \to 800$ that it has the expected asymptotic scaling

## MERA diagram
Skipped due to time constraints